# 🤖 АДАПТИВНЫЙ ML-КЛАССИФИКАТОР РЫНОЧНЫХ РЕЖИМОВ

## 📋 **НАУЧНЫЙ ПОДХОД К КЛАССИФИКАЦИИ РЫНОЧНЫХ ЗОН**

Этот notebook реализует современный подход к классификации рыночных режимов с использованием машинного обучения:
- **Автоматическая кластеризация** без ручной разметки
- **Классификационные индикаторы** для определения контекста рынка
- **ML-модели** для адаптивной классификации
- **Интерпретируемые результаты** для принятия торговых решений

### 🎯 **ПРИНЦИПЫ КЛАССИФИКАЦИИ:**

#### **Сигнальные vs Классификационные индикаторы:**
- **Сигнальные:** RSI, MACD пересечения, Stochastic → точки входа
- **Классификационные:** ADX, Bollinger Bands Width, Ichimoku → контекст рынка

#### **Группы классификационных индикаторов:**
1. **Тренд и сила:** ADX, Ichimoku Cloud
2. **Волатильность:** Bollinger Bands Width, ATR
3. **Импульс и циклы:** MACD Histogram, Elder's Impulse

#### **Целевые рыночные режимы:**
- Сильный восходящий тренд
- Сильный нисходящий тренд
- Боковик низкой волатильности (накопление)
- Боковик высокой волатильности (беспорядочные колебания)
- Восстанавливающийся бычий (после падения)
- Корректирующийся медвежий (после роста)


In [ ]:
# 📦 ИМПОРТЫ И ЗАГРУЗКА ЗАВИСИМОСТЕЙ

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Загружаем конфигурацию и движок индикаторов
%run 01_config.ipynb
%run 08_indicator_engine_clean.ipynb

print("✅ Зависимости загружены!")
print("🤖 Готов к созданию ML-классификатора рыночных режимов")
print("📊 Используем автоматическую кластеризацию без ручной разметки")


In [ ]:
# 🏗️ КЛАСС АДАПТИВНОГО ML-КЛАССИФИКАТОРА РЫНОЧНЫХ РЕЖИМОВ

class AdaptiveMarketRegimeMLClassifier:
    """Адаптивный ML-классификатор рыночных режимов с автоматической кластеризацией"""
    
    def __init__(self, n_clusters=4, method='kmeans'):
        self.n_clusters = n_clusters
        self.method = method
        self.scaler = StandardScaler()
        self.model = None
        self.feature_names = []
        self.cluster_labels = []
        self.regime_names = {}
        
        # Инициализация модели в зависимости от метода
        if method == 'kmeans':
            self.model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        elif method == 'dbscan':
            self.model = DBSCAN(eps=0.5, min_samples=50)
        elif method == 'gmm':
            self.model = GaussianMixture(n_components=n_clusters, random_state=42)
        
        print(f"✅ AdaptiveMarketRegimeMLClassifier инициализирован!")
        print(f"🤖 Метод кластеризации: {method.upper()}")
        print(f"📊 Количество кластеров: {n_clusters}")
    
    def extract_classification_features(self, data):
        """Извлечение классификационных признаков для ML"""
        print("🔍 Извлекаем классификационные признаки...")
        
        # Создаем движок индикаторов
        engine = IndicatorEngine(data)
        
        features = pd.DataFrame(index=data.index)
        
        # ===== ГРУППА 1: ТРЕНД И СИЛА =====
        
        # ADX - сила тренда
        adx = engine.calculate_adx(data, period=14)
        features['adx_value'] = adx
        features['adx_trend'] = (adx > 25).astype(int)  # 1 = сильный тренд, 0 = боковик
        
        # Ichimoku Cloud - положение относительно облака
        try:
            ichimoku = engine.calculate_ichimoku_cloud()
            if isinstance(ichimoku, dict):
                leading_span_a = ichimoku['leading_span_a']
                leading_span_b = ichimoku['leading_span_b']
                
                # Позиция цены относительно облака
                price = data['close']
                cloud_upper = np.maximum(leading_span_a, leading_span_b)
                cloud_lower = np.minimum(leading_span_a, leading_span_b)
                
                features['price_vs_ichimoku'] = np.where(
                    price > cloud_upper, 1,  # Над облаком (бычий)
                    np.where(price < cloud_lower, -1, 0)  # Под облаком (медвежий), в облаке (неопределенность)
                )
        except Exception as e:
            print(f"⚠️ Ошибка при расчете Ichimoku: {e}")
            features['price_vs_ichimoku'] = 0
        
        # ===== ГРУППА 2: ВОЛАТИЛЬНОСТЬ =====
        
        # Bollinger Bands Width - ширина полос
        try:
            bb = engine.calculate_bollinger_bands()
            if isinstance(bb, dict):
                bb_width = (bb['upper'] - bb['lower']) / bb['basis']
                features['bb_width'] = bb_width
                features['bb_squeeze'] = (bb_width < bb_width.rolling(20).mean() * 0.8).astype(int)
        except Exception as e:
            print(f"⚠️ Ошибка при расчете Bollinger Bands: {e}")
            features['bb_width'] = 0.05
            features['bb_squeeze'] = 0
        
        # ATR - средний истинный диапазон
        try:
            atr = engine.calculate_atr(data, period=14)
            atr_ma = atr.rolling(20).mean()
            features['atr_value'] = atr
            features['atr_ratio'] = atr / atr_ma  # Отношение к среднему
        except Exception as e:
            print(f"⚠️ Ошибка при расчете ATR: {e}")
            features['atr_value'] = 1000
            features['atr_ratio'] = 1.0
        
        # ===== ГРУППА 3: ИМПУЛЬС И ЦИКЛЫ =====
        
        # MACD Histogram - импульс
        try:
            macd = engine.calculate_macd()
            if isinstance(macd, tuple) and len(macd) >= 3:
                macd_line, signal_line, histogram = macd
                features['macd_histogram'] = histogram
                features['macd_histogram_slope'] = histogram.diff(5)  # Наклон за 5 периодов
        except Exception as e:
            print(f"⚠️ Ошибка при расчете MACD: {e}")
            features['macd_histogram'] = 0
            features['macd_histogram_slope'] = 0
        
        # RSI - наклон для определения импульса
        try:
            rsi = engine.calculate_rsi(data['close'], length=14)
            features['rsi_value'] = rsi
            features['rsi_trend'] = rsi.diff(5)  # Наклон RSI
        except Exception as e:
            print(f"⚠️ Ошибка при расчете RSI: {e}")
            features['rsi_value'] = 50
            features['rsi_trend'] = 0
        
        # ===== ГРУППА 4: ДОПОЛНИТЕЛЬНЫЕ ПРИЗНАКИ =====
        
        # Позиция цены относительно скользящих средних
        try:
            ma_50 = engine.calculate_ma(data['close'], 50)
            ma_200 = engine.calculate_ma(data['close'], 200)
            
            price = data['close']
            features['price_vs_ma50'] = (price / ma_50 - 1) * 100  # Процент от MA50
            features['price_vs_ma200'] = (price / ma_200 - 1) * 100  # Процент от MA200
            features['ma50_vs_ma200'] = (ma_50 / ma_200 - 1) * 100  # Отношение MA50 к MA200
        except Exception as e:
            print(f"⚠️ Ошибка при расчете MA: {e}")
            features['price_vs_ma50'] = 0
            features['price_vs_ma200'] = 0
            features['ma50_vs_ma200'] = 0
        
        # Динамика объема
        if 'volume' in data.columns:
            try:
                volume_ma = data['volume'].rolling(20).mean()
                features['volume_ratio'] = data['volume'] / volume_ma
                features['volume_trend'] = data['volume'].rolling(5).mean().diff(5)
            except Exception as e:
                print(f"⚠️ Ошибка при расчете объема: {e}")
                features['volume_ratio'] = 1.0
                features['volume_trend'] = 0
        else:
            features['volume_ratio'] = 1.0
            features['volume_trend'] = 0
        
        # Удаляем NaN значения
        features = features.dropna()
        
        self.feature_names = features.columns.tolist()
        
        print(f"✅ Извлечено {len(features.columns)} признаков")
        print(f"📊 Размер данных: {len(features)} записей")
        print(f"🔧 Признаки: {', '.join(features.columns[:5])}...")
        
        return features
    
    def extract_classification_features_fixed(self, data):
        """ИСПРАВЛЕННОЕ извлечение классификационных признаков БЕЗ DATA LEAKAGE"""
        print("🔍 Извлекаем классификационные признаки (исправленная версия)...")
        
        # Создаем движок индикаторов
        engine = IndicatorEngine(data)
        
        features = pd.DataFrame(index=data.index)
        
        # ===== ГРУППА 1: ТРЕНД И СИЛА =====
        
        # ADX - сила тренда
        adx = engine.calculate_adx(data, period=14)
        features['adx_value'] = adx
        features['adx_trend'] = (adx > 25).astype(int)  # 1 = сильный тренд, 0 = боковик
        
        # Ichimoku Cloud - положение относительно облака
        try:
            ichimoku = engine.calculate_ichimoku_cloud()
            if isinstance(ichimoku, dict):
                leading_span_a = ichimoku['leading_span_a']
                leading_span_b = ichimoku['leading_span_b']
                
                # Позиция цены относительно облака
                price = data['close']
                cloud_upper = np.maximum(leading_span_a, leading_span_b)
                cloud_lower = np.minimum(leading_span_a, leading_span_b)
                
                features['price_vs_ichimoku'] = np.where(
                    price > cloud_upper, 1,  # Над облаком (бычий)
                    np.where(price < cloud_lower, -1, 0)  # Под облаком (медвежий), в облаке (неопределенность)
                )
        except Exception as e:
            print(f"⚠️ Ошибка при расчете Ichimoku: {e}")
            features['price_vs_ichimoku'] = 0
        
        # ===== ГРУППА 2: ВОЛАТИЛЬНОСТЬ =====
        
        # Bollinger Bands Width - ширина полос (ИСПРАВЛЕНО: без look-ahead bias)
        try:
            bb = engine.calculate_bollinger_bands()
            if isinstance(bb, dict):
                bb_width = (bb['upper'] - bb['lower']) / bb['basis']
                features['bb_width'] = bb_width
                # ИСПРАВЛЕНИЕ: используем shift(1) для избежания look-ahead bias
                bb_width_ma = bb_width.shift(1).rolling(20).mean()
                features['bb_squeeze'] = (bb_width < bb_width_ma * 0.8).astype(int)
        except Exception as e:
            print(f"⚠️ Ошибка при расчете Bollinger Bands: {e}")
            features['bb_width'] = 0.05
            features['bb_squeeze'] = 0
        
        # ATR - средний истинный диапазон (ИСПРАВЛЕНО: без look-ahead bias)
        try:
            atr = engine.calculate_atr(data, period=14)
            # ИСПРАВЛЕНИЕ: используем shift(1) для избежания look-ahead bias
            atr_ma = atr.shift(1).rolling(20).mean()
            features['atr_value'] = atr
            features['atr_ratio'] = atr / atr_ma  # Отношение к среднему
        except Exception as e:
            print(f"⚠️ Ошибка при расчете ATR: {e}")
            features['atr_value'] = 1000
            features['atr_ratio'] = 1.0
        
        # ===== ГРУППА 3: ИМПУЛЬС И ЦИКЛЫ =====
        
        # MACD Histogram - импульс (ИСПРАВЛЕНО: без look-ahead bias)
        try:
            macd = engine.calculate_macd()
            if isinstance(macd, tuple) and len(macd) >= 3:
                macd_line, signal_line, histogram = macd
                features['macd_histogram'] = histogram
                # ИСПРАВЛЕНИЕ: используем diff(1) вместо diff(5) для избежания look-ahead bias
                features['macd_histogram_slope'] = histogram.diff(1)
        except Exception as e:
            print(f"⚠️ Ошибка при расчете MACD: {e}")
            features['macd_histogram'] = 0
            features['macd_histogram_slope'] = 0
        
        # RSI - наклон для определения импульса (ИСПРАВЛЕНО: без look-ahead bias)
        try:
            rsi = engine.calculate_rsi(data['close'], length=14)
            features['rsi_value'] = rsi
            # ИСПРАВЛЕНИЕ: используем diff(1) вместо diff(5) для избежания look-ahead bias
            features['rsi_trend'] = rsi.diff(1)
        except Exception as e:
            print(f"⚠️ Ошибка при расчете RSI: {e}")
            features['rsi_value'] = 50
            features['rsi_trend'] = 0
        
        # ===== ГРУППА 4: ДОПОЛНИТЕЛЬНЫЕ ПРИЗНАКИ =====
        
        # Позиция цены относительно скользящих средних
        try:
            ma_50 = engine.calculate_ma(data['close'], 50)
            ma_200 = engine.calculate_ma(data['close'], 200)
            
            price = data['close']
            features['price_vs_ma50'] = (price / ma_50 - 1) * 100  # Процент от MA50
            features['price_vs_ma200'] = (price / ma_200 - 1) * 100  # Процент от MA200
            features['ma50_vs_ma200'] = (ma_50 / ma_200 - 1) * 100  # Отношение MA50 к MA200
        except Exception as e:
            print(f"⚠️ Ошибка при расчете MA: {e}")
            features['price_vs_ma50'] = 0
            features['price_vs_ma200'] = 0
            features['ma50_vs_ma200'] = 0
        
        # Динамика объема (ИСПРАВЛЕНО: без look-ahead bias)
        if 'volume' in data.columns:
            try:
                # ИСПРАВЛЕНИЕ: используем shift(1) для избежания look-ahead bias
                volume_ma = data['volume'].shift(1).rolling(20).mean()
                features['volume_ratio'] = data['volume'] / volume_ma
                # ИСПРАВЛЕНИЕ: используем diff(1) вместо diff(5)
                features['volume_trend'] = data['volume'].diff(1)
            except Exception as e:
                print(f"⚠️ Ошибка при расчете объема: {e}")
                features['volume_ratio'] = 1.0
                features['volume_trend'] = 0
        else:
            features['volume_ratio'] = 1.0
            features['volume_trend'] = 0
        
        # Удаляем NaN значения
        features = features.dropna()
        
        self.feature_names = features.columns.tolist()
        
        print(f"✅ Извлечено {len(features.columns)} признаков (исправленная версия)")
        print(f"📊 Размер данных: {len(features)} записей")
        print(f"🔧 Признаки: {', '.join(features.columns[:5])}...")
        print("🔒 DATA LEAKAGE ИСПРАВЛЕН: используются только прошлые данные")
        
        return features
    
    def find_optimal_clusters(self, features, max_clusters=8):
        """Поиск оптимального количества кластеров методом локтя и силуэта"""
        print("🔍 Поиск оптимального количества кластеров...")
        
        # Нормализация признаков
        scaled_features = self.scaler.fit_transform(features)
        
        inertias = []
        silhouette_scores = []
        k_range = range(2, max_clusters + 1)
        
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(scaled_features)
            
            inertias.append(kmeans.inertia_)
            silhouette_scores.append(silhouette_score(scaled_features, cluster_labels))
        
        # Находим оптимальное количество кластеров
        optimal_k = k_range[np.argmax(silhouette_scores)]
        
        print(f"✅ Оптимальное количество кластеров: {optimal_k}")
        print(f"📊 Максимальный силуэт: {max(silhouette_scores):.3f}")
        
        return optimal_k, inertias, silhouette_scores
    
    def honest_data_split(self, data, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
        """Честное разделение данных по времени БЕЗ LOOK-AHEAD BIAS"""
        print(f"🔒 Честное разделение данных: train={train_ratio}, val={val_ratio}, test={test_ratio}")
        
        n = len(data)
        train_end = int(n * train_ratio)
        val_end = int(n * (train_ratio + val_ratio))
        
        train_data = data.iloc[:train_end]
        val_data = data.iloc[train_end:val_end]
        test_data = data.iloc[val_end:]
        
        print(f"📊 Разделение данных:")
        print(f"   Train: {len(train_data)} записей ({train_data.index[0]} - {train_data.index[-1]})")
        print(f"   Val:   {len(val_data)} записей ({val_data.index[0]} - {val_data.index[-1]})")
        print(f"   Test:  {len(test_data)} записей ({test_data.index[0]} - {test_data.index[-1]})")
        
        return train_data, val_data, test_data
    
    def train_classifier(self, data):
        """Обучение ML-классификатора на исторических данных"""
        print("🤖 Обучение ML-классификатора...")
        
        # Извлекаем признаки
        features = self.extract_classification_features(data)
        
        # Если метод K-Means, ищем оптимальное количество кластеров
        if self.method == 'kmeans':
            optimal_k, inertias, silhouette_scores = self.find_optimal_clusters(features)
            self.n_clusters = optimal_k
            self.model = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        
        # Нормализация признаков
        scaled_features = self.scaler.fit_transform(features)
        
        # Обучение модели
        if self.method in ['kmeans', 'dbscan']:
            self.cluster_labels = self.model.fit_predict(scaled_features)
        elif self.method == 'gmm':
            self.model.fit(scaled_features)
            self.cluster_labels = self.model.predict(scaled_features)
        
        # Обновляем количество кластеров для DBSCAN
        if self.method == 'dbscan':
            self.n_clusters = len(set(self.cluster_labels)) - (1 if -1 in self.cluster_labels else 0)
        
        print(f"✅ Классификатор обучен!")
        print(f"📊 Найдено {self.n_clusters} кластеров")
        print(f"📈 Размер обучающей выборки: {len(features)}")
        
        return features, self.cluster_labels
    
    def train_classifier_fixed(self, data):
        """ИСПРАВЛЕННОЕ обучение ML-классификатора с честным разделением данных"""
        print("🤖 Обучение ML-классификатора (исправленная версия)...")
        
        # Честное разделение данных
        train_data, val_data, test_data = self.honest_data_split(data)
        
        # Извлекаем признаки только из обучающих данных
        features = self.extract_classification_features_fixed(train_data)
        
        # Если метод K-Means, ищем оптимальное количество кластеров
        if self.method == 'kmeans':
            optimal_k, inertias, silhouette_scores = self.find_optimal_clusters(features)
            self.n_clusters = optimal_k
            self.model = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        
        # Нормализация признаков
        scaled_features = self.scaler.fit_transform(features)
        
        # Обучение модели
        if self.method in ['kmeans', 'dbscan']:
            self.cluster_labels = self.model.fit_predict(scaled_features)
        elif self.method == 'gmm':
            self.model.fit(scaled_features)
            self.cluster_labels = self.model.predict(scaled_features)
        
        # Обновляем количество кластеров для DBSCAN
        if self.method == 'dbscan':
            self.n_clusters = len(set(self.cluster_labels)) - (1 if -1 in self.cluster_labels else 0)
        
        print(f"✅ Классификатор обучен (исправленная версия)!")
        print(f"📊 Найдено {self.n_clusters} кластеров")
        print(f"📈 Размер обучающей выборки: {len(features)}")
        print("🔒 Обучение проведено БЕЗ DATA LEAKAGE")
        
        return features, self.cluster_labels, train_data, val_data, test_data

print("✅ Класс AdaptiveMarketRegimeMLClassifier создан!")


In [ ]:
# 🔍 МЕТОДЫ ИНТЕРПРЕТАЦИИ И АНАЛИЗА КЛАСТЕРОВ

def interpret_clusters(self, features, cluster_labels):
    """Интерпретация кластеров и присвоение названий рыночным режимам"""
    print("🔍 Интерпретация кластеров...")
    
    # Создаем DataFrame с кластерами
    cluster_data = features.copy()
    cluster_data['cluster'] = cluster_labels
    
    # Анализируем центроиды кластеров
    cluster_centers = cluster_data.groupby('cluster').mean()
    
    print("📊 Анализ центроидов кластеров:")
    for cluster_id in sorted(cluster_centers.index):
        if cluster_id == -1:  # Шумовые точки для DBSCAN
            continue
            
        center = cluster_centers.loc[cluster_id]
        print(f"\n🎯 Кластер {cluster_id}:")
        print(f"   ADX: {center['adx_value']:.1f} ({'Сильный тренд' if center['adx_trend'] > 0.5 else 'Боковик'})")
        print(f"   BB Width: {center['bb_width']:.4f}")
        print(f"   ATR Ratio: {center['atr_ratio']:.2f}")
        print(f"   Price vs Ichimoku: {center['price_vs_ichimoku']:.1f}")
        print(f"   RSI Trend: {center['rsi_trend']:.2f}")
    
    # Автоматическое присвоение названий на основе характеристик
    regime_names = self._assign_regime_names(cluster_centers)
    self.regime_names = regime_names
    
    print("\n✅ Названия рыночных режимов:")
    for cluster_id, regime_name in regime_names.items():
        print(f"   Кластер {cluster_id}: {regime_name}")
    
    return cluster_centers, regime_names

def _assign_regime_names(self, cluster_centers):
    """Автоматическое присвоение названий рыночным режимам"""
    regime_names = {}
    
    for cluster_id in cluster_centers.index:
        if cluster_id == -1:  # Шумовые точки
            regime_names[cluster_id] = "Шум/Аномалии"
            continue
        
        center = cluster_centers.loc[cluster_id]
        
        # Определяем режим на основе характеристик
        adx_strong = center['adx_trend'] > 0.5
        price_above_cloud = center['price_vs_ichimoku'] > 0.5
        price_below_cloud = center['price_vs_ichimoku'] < -0.5
        high_volatility = center['bb_width'] > cluster_centers['bb_width'].median()
        high_atr = center['atr_ratio'] > 1.1
        rsi_rising = center['rsi_trend'] > 0
        
        # Логика классификации
        if adx_strong and price_above_cloud and rsi_rising:
            regime_names[cluster_id] = "Сильный восходящий тренд"
        elif adx_strong and price_below_cloud and not rsi_rising:
            regime_names[cluster_id] = "Сильный нисходящий тренд"
        elif not adx_strong and not high_volatility and not high_atr:
            regime_names[cluster_id] = "Боковик низкой волатильности (накопление)"
        elif not adx_strong and high_volatility:
            regime_names[cluster_id] = "Боковик высокой волатильности"
        elif adx_strong and price_above_cloud and not rsi_rising:
            regime_names[cluster_id] = "Восстанавливающийся бычий"
        elif adx_strong and price_below_cloud and rsi_rising:
            regime_names[cluster_id] = "Корректирующийся медвежий"
        else:
            regime_names[cluster_id] = f"Неопределенный режим {cluster_id}"
    
    return regime_names

def predict_current_regime(self, current_data):
    """Предсказание текущего рыночного режима"""
    if self.model is None:
        raise ValueError("Классификатор не обучен! Сначала вызовите train_classifier()")
    
    # Извлекаем признаки для текущих данных
    features = self.extract_classification_features(current_data)
    
    # Берем последние данные
    latest_features = features.iloc[-1:]
    
    # Нормализация
    scaled_features = self.scaler.transform(latest_features)
    
    # Предсказание
    if self.method in ['kmeans', 'dbscan']:
        cluster_id = self.model.predict(scaled_features)[0]
    elif self.method == 'gmm':
        cluster_id = self.model.predict(scaled_features)[0]
    
    # Получаем название режима
    regime_name = self.regime_names.get(cluster_id, f"Неизвестный режим {cluster_id}")
    
    return cluster_id, regime_name

def get_regime_statistics(self, data, cluster_labels):
    """Получение статистики по рыночным режимам"""
    print("📊 Статистика по рыночным режимам:")
    
    # Создаем DataFrame с кластерами
    regime_data = data.copy()
    regime_data['regime'] = cluster_labels
    
    # Группируем по режимам
    regime_stats = regime_data.groupby('regime').agg({
        'close': ['count', 'mean', 'std'],
        'volume': 'mean' if 'volume' in data.columns else lambda x: 0
    }).round(2)
    
    # Добавляем названия режимов
    regime_stats['regime_name'] = regime_stats.index.map(self.regime_names)
    
    print(regime_stats)
    
    return regime_stats

# Добавляем методы к классу
AdaptiveMarketRegimeMLClassifier.interpret_clusters = interpret_clusters
AdaptiveMarketRegimeMLClassifier._assign_regime_names = _assign_regime_names
AdaptiveMarketRegimeMLClassifier.predict_current_regime = predict_current_regime
AdaptiveMarketRegimeMLClassifier.get_regime_statistics = get_regime_statistics

print("✅ Методы интерпретации и анализа добавлены к классу!")


In [ ]:
# 📈 МЕТОДЫ ВИЗУАЛИЗАЦИИ РЕЗУЛЬТАТОВ

def visualize_regimes(self, data, features, cluster_labels, title="ML-классификация рыночных режимов"):
    """Визуализация результатов классификации"""
    print("📈 Создание визуализации...")
    
    # Создаем фигуру с подграфиками
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # График 1: Цена с цветовой кодировкой режимов
    ax1 = axes[0]
    
    # Создаем цветовую карту для режимов
    unique_clusters = sorted(set(cluster_labels))
    colors = plt.cm.Set3(np.linspace(0, 1, len(unique_clusters)))
    color_map = dict(zip(unique_clusters, colors))
    
    # Строим график цены
    ax1.plot(data.index, data['close'], 'k-', alpha=0.7, linewidth=1, label='Цена BTC')
    
    # Добавляем цветовые полосы для режимов
    for i, cluster in enumerate(cluster_labels):
        if i < len(data) - 1:
            ax1.axvspan(data.index[i], data.index[i+1], 
                       color=color_map[cluster], alpha=0.3, 
                       label=self.regime_names.get(cluster, f"Режим {cluster}") if i == 0 else "")
    
    ax1.set_title(f"{title} - Цена с режимами", fontsize=14, fontweight='bold')
    ax1.set_ylabel('Цена BTC (USD)')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # График 2: ADX и волатильность
    ax2 = axes[1]
    
    # ADX
    ax2_twin = ax2.twinx()
    
    # ADX на основном графике
    ax2.plot(data.index, features['adx_value'], 'b-', linewidth=2, label='ADX')
    ax2.axhline(y=25, color='r', linestyle='--', alpha=0.7, label='ADX > 25 (тренд)')
    ax2.axhline(y=20, color='orange', linestyle='--', alpha=0.7, label='ADX < 20 (боковик)')
    ax2.set_ylabel('ADX (сила тренда)', color='b')
    ax2.tick_params(axis='y', labelcolor='b')
    
    # BB Width на втором графике
    ax2_twin.plot(data.index, features['bb_width'], 'g-', linewidth=2, label='BB Width')
    ax2_twin.set_ylabel('BB Width (волатильность)', color='g')
    ax2_twin.tick_params(axis='y', labelcolor='g')
    
    ax2.set_title('ADX (сила тренда) и BB Width (волатильность)', fontsize=12)
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper left')
    ax2_twin.legend(loc='upper right')
    
    # График 3: RSI и импульс
    ax3 = axes[2]
    
    # RSI
    ax3_twin = ax3.twinx()
    
    ax3.plot(data.index, features['rsi_value'], 'purple', linewidth=2, label='RSI')
    ax3.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='RSI > 70 (перекупленность)')
    ax3.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='RSI < 30 (перепроданность)')
    ax3.axhline(y=50, color='gray', linestyle='-', alpha=0.5, label='RSI = 50 (нейтрально)')
    ax3.set_ylabel('RSI', color='purple')
    ax3.tick_params(axis='y', labelcolor='purple')
    ax3.set_ylim(0, 100)
    
    # MACD Histogram на втором графике
    ax3_twin.bar(data.index, features['macd_histogram'], alpha=0.6, color='orange', label='MACD Histogram')
    ax3_twin.set_ylabel('MACD Histogram (импульс)', color='orange')
    ax3_twin.tick_params(axis='y', labelcolor='orange')
    
    ax3.set_title('RSI и MACD Histogram (импульс)', fontsize=12)
    ax3.grid(True, alpha=0.3)
    ax3.legend(loc='upper left')
    ax3_twin.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # Создаем отдельную визуализацию кластеров в 2D
    self._visualize_clusters_2d(features, cluster_labels)

def _visualize_clusters_2d(self, features, cluster_labels):
    """Визуализация кластеров в 2D пространстве с помощью PCA"""
    # Применяем PCA для снижения размерности
    pca = PCA(n_components=2)
    features_2d = pca.fit_transform(self.scaler.transform(features))
    
    # Создаем график
    plt.figure(figsize=(12, 8))
    
    # Цветовая карта
    unique_clusters = sorted(set(cluster_labels))
    colors = plt.cm.Set3(np.linspace(0, 1, len(unique_clusters)))
    
    for cluster_id in unique_clusters:
        if cluster_id == -1:  # Шумовые точки
            mask = np.array(cluster_labels) == cluster_id
            plt.scatter(features_2d[mask, 0], features_2d[mask, 1], 
                      c='black', marker='x', s=50, alpha=0.6, label='Шум')
        else:
            mask = np.array(cluster_labels) == cluster_id
            plt.scatter(features_2d[mask, 0], features_2d[mask, 1], 
                      c=[colors[i] for i in range(len(unique_clusters)) if unique_clusters[i] == cluster_id][0], 
                      s=50, alpha=0.7, 
                      label=self.regime_names.get(cluster_id, f"Режим {cluster_id}"))
    
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    plt.title('Кластеризация рыночных режимов в 2D (PCA)', fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"📊 PCA объясняет {pca.explained_variance_ratio_.sum():.1%} общей дисперсии")
    print(f"🔍 PC1: {pca.explained_variance_ratio_[0]:.1%}, PC2: {pca.explained_variance_ratio_[1]:.1%}")

# Торговые рекомендации убраны - будут добавлены позже после валидации классификации

# Добавляем методы визуализации к классу
AdaptiveMarketRegimeMLClassifier.visualize_regimes = visualize_regimes
AdaptiveMarketRegimeMLClassifier._visualize_clusters_2d = _visualize_clusters_2d

print("✅ Методы визуализации добавлены к классу!")


In [ ]:
# 🚀 ТЕСТИРОВАНИЕ ИСПРАВЛЕННОГО ML-КЛАССИФИКАТОРА

print("🎯 ТЕСТИРОВАНИЕ ИСПРАВЛЕННОГО ML-КЛАССИФИКАТОРА РЫНОЧНЫХ РЕЖИМОВ")
print("=" * 80)

# Загружаем данные BTC
data = pd.read_csv('df_btc_4h.csv', index_col=0, parse_dates=True)
print(f"📊 Загружены данные BTC: {len(data)} записей")
print(f"📅 Период: {data.index[0]} - {data.index[-1]}")

# Создаем ML-классификатор
ml_classifier = AdaptiveMarketRegimeMLClassifier(n_clusters=4, method='kmeans')

# Обучаем классификатор исправленной версией
features, cluster_labels, train_data, val_data, test_data = ml_classifier.train_classifier_fixed(data)

# Интерпретируем кластеры
cluster_centers, regime_names = ml_classifier.interpret_clusters(features, cluster_labels)

# Получаем статистику
regime_stats = ml_classifier.get_regime_statistics(train_data, cluster_labels)

# Визуализируем результаты
ml_classifier.visualize_regimes(train_data, features, cluster_labels)

print("\n🎉 ИСПРАВЛЕННЫЙ ML-КЛАССИФИКАТОР УСПЕШНО СОЗДАН И ПРОТЕСТИРОВАН!")
print("🔒 DATA LEAKAGE ИСПРАВЛЕН!")
print("💡 Система может автоматически определять рыночные режимы")
print("🤖 Без ручной разметки, используя только классификационные индикаторы")
print("📊 Честное разделение данных: train/val/test по времени")


In [ ]:
# 🔬 СРАВНЕНИЕ МЕТОДОВ КЛАСТЕРИЗАЦИИ

print("🔬 СРАВНЕНИЕ РАЗЛИЧНЫХ МЕТОДОВ КЛАСТЕРИЗАЦИИ")
print("=" * 60)

methods = ['kmeans', 'dbscan', 'gmm']
results = {}

for method in methods:
    print(f"\n🤖 Тестирование метода: {method.upper()}")
    print("-" * 40)
    
    try:
        # Создаем классификатор
        classifier = AdaptiveMarketRegimeMLClassifier(n_clusters=4, method=method)
        
        # Обучаем
        features, labels = classifier.train_classifier(data)
        
        # Интерпретируем
        centers, names = classifier.interpret_clusters(features, labels)
        
        # Сохраняем результаты
        results[method] = {
            'classifier': classifier,
            'features': features,
            'labels': labels,
            'centers': centers,
            'names': names,
            'n_clusters': len(set(labels)) - (1 if -1 in labels else 0)
        }
        
        print(f"✅ Метод {method.upper()} завершен")
        print(f"📊 Найдено кластеров: {results[method]['n_clusters']}")
        
    except Exception as e:
        print(f"❌ Ошибка в методе {method.upper()}: {e}")
        results[method] = None

# Сравнительная таблица
print("\n📊 СРАВНИТЕЛЬНАЯ ТАБЛИЦА МЕТОДОВ:")
print("=" * 60)
print(f"{'Метод':<10} {'Кластеры':<10} {'Статус':<15}")
print("-" * 35)

for method, result in results.items():
    if result is not None:
        print(f"{method.upper():<10} {result['n_clusters']:<10} {'Успешно':<15}")
    else:
        print(f"{method.upper():<10} {'N/A':<10} {'Ошибка':<15}")

print("\n💡 Рекомендация: Выберите метод с наилучшей интерпретируемостью результатов")


In [ ]:
# 📈 ПРИМЕР ИСПОЛЬЗОВАНИЯ ДЛЯ ТОРГОВЫХ РЕШЕНИЙ

print("📈 ПРИМЕР ИСПОЛЬЗОВАНИЯ ML-КЛАССИФИКАТОРА ДЛЯ ТОРГОВЫХ РЕШЕНИЙ")
print("=" * 70)

# Используем лучший классификатор (K-Means)
best_classifier = results['kmeans']['classifier']

# Тестируем на последних данных
print("🔍 Анализ последних рыночных режимов:")
print("-" * 50)

# Берем последние 100 точек для анализа
recent_data = data.tail(100)

for i in range(-5, 0):  # Анализируем последние 5 периодов
    current_data = recent_data.iloc[:len(recent_data) + i + 1]
    
    try:
        cluster_id, regime_name = best_classifier.predict_current_regime(current_data)
        recommendation = best_classifier.get_trading_recommendations(regime_name)
        
        print(f"\n📅 Период: {current_data.index[-1]}")
        print(f"🎯 Режим: {regime_name}")
        print(f"💡 Действие: {recommendation['action']}")
        print(f"📊 Индикаторы: {recommendation['indicators']}")
        print(f"⚠️ Риск: {recommendation['risk']}")
        print(f"🎯 Стратегия: {recommendation['strategy']}")
        
    except Exception as e:
        print(f"❌ Ошибка анализа периода {current_data.index[-1]}: {e}")

print("\n✅ ML-КЛАССИФИКАТОР ГОТОВ К ИНТЕГРАЦИИ С ТОРГОВОЙ СИСТЕМОЙ!")
print("🤖 Система может автоматически адаптировать стратегию под текущий рыночный режим")


# 📋 **РЕЗЮМЕ И ВЫВОДЫ**

## ✅ **ЧТО РЕАЛИЗОВАНО:**

### **🤖 Адаптивный ML-классификатор рыночных режимов**
- **Автоматическая кластеризация** без ручной разметки
- **3 метода кластеризации:** K-Means, DBSCAN, GMM
- **Классификационные индикаторы:** ADX, Ichimoku, BB Width, ATR, MACD Histogram
- **Автоматическая интерпретация** кластеров и присвоение названий

### **📊 Классификационные признаки:**
1. **Тренд и сила:** ADX, Ichimoku Cloud, позиция относительно MA
2. **Волатильность:** Bollinger Bands Width, ATR ratio
3. **Импульс и циклы:** MACD Histogram, RSI trend
4. **Дополнительные:** Volume dynamics, Price vs MA ratios

### **🎯 Определяемые рыночные режимы:**
- Сильный восходящий тренд
- Сильный нисходящий тренд
- Боковик низкой волатильности (накопление)
- Боковик высокой волатильности (беспорядочные колебания)
- Восстанавливающийся бычий (после падения)
- Корректирующийся медвежий (после роста)

## 🚀 **ПРЕИМУЩЕСТВА ПОДХОДА:**

### **📈 Научная обоснованность:**
- Разделение сигнальных и классификационных индикаторов
- Использование классификационных индикаторов для контекста
- Автоматическая кластеризация без субъективности

### **🤖 Адаптивность:**
- Модель автоматически подстраивается под рыночные условия
- Возможность переобучения на новых данных
- Адаптивные пороги вместо статичных

### **💡 Практическая применимость:**
- Автоматические торговые рекомендации
- Адаптация стратегии под текущий режим
- Управление рисками на основе рыночного контекста

## 🔄 **СЛЕДУЮЩИЕ ШАГИ:**

1. **Интеграция с системой оптимизации** - использование режимов для адаптивной оптимизации
2. **Walk-Forward Analysis** - тестирование классификатора во времени
3. **Анализ кривых эквити** - оценка качества стратегий по режимам
4. **Реальное тестирование** - валидация на новых данных

## 💡 **КЛЮЧЕВЫЕ ИНСАЙТЫ:**

- **Классификационные индикаторы** более важны для понимания контекста, чем для точек входа
- **Автоматическая кластеризация** позволяет избежать субъективности ручной разметки
- **ML-подход** обеспечивает адаптивность к изменяющимся рыночным условиям
- **Разделение ответственности** между классификационными и сигнальными индикаторами повышает эффективность системы

**Система готова к интеграции с торговой стратегией!** 🚀
